In [1]:
!pip install -q gradio transformers scikit-learn

In [2]:
import joblib
import gradio as gr
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from transformers import pipeline

In [3]:
data = {
    'return': [
        "How do I return a product?",
        "I want to send something back",
        "Return policy details",
        "Refund for returned item"
    ],
    'delivery': [
        "Where is my package?",
        "My order is late",
        "Delivery status of my order",
        "Track my shipment"
    ],
    'technical': [
        "My Kindle won't start",
        "How do I connect Alexa to Wi-Fi?",
        "App not working on Firestick",
        "Device not responding"
    ],
    'general': [
        "How do I change my address?",
        "Can I change payment method?",
        "Update account settings",
        "Need help with my profile"
    ]
}

In [5]:
X, y = [], []
for intent, samples in data.items():
    X.extend(samples)
    y.extend([intent] * len(samples))

In [6]:
vectorizer = TfidfVectorizer()
X_vec = vectorizer.fit_transform(X)
clf = LogisticRegression()
clf.fit(X_vec, y)

LogisticRegression()

In [7]:
joblib.dump(clf, "intent_classifier.pkl")
joblib.dump(vectorizer, "vectorizer.pkl")

['vectorizer.pkl']

In [8]:
def returns_agent(query):
    return "You can return your product from the 'Your Orders' section. Need help initiating it?"

def delivery_agent(query):
    return "Please visit 'Your Orders' to track the package. Do you need a delivery update?"

def technical_agent(query):
    return "Let me help you with a technical issue. Try restarting the device or checking your connection."

def general_agent(query):
    return "Sure, you can update your info in Account Settings. Anything else I can help with?"

def fallback_agent(query):
    return "I'm not sure I understood that. Could you please rephrase your question or contact support?"

In [9]:
def master_agent(query):
    clf = joblib.load("intent_classifier.pkl")
    vec = joblib.load("vectorizer.pkl")
    X_input = vec.transform([query])
    intent = clf.predict(X_input)[0]

    if intent == "return":
        return returns_agent(query)
    elif intent == "delivery":
        return delivery_agent(query)
    elif intent == "technical":
        return technical_agent(query)
    elif intent == "general":
        return general_agent(query)
    else:
        return fallback_agent(query)

In [10]:
sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


In [11]:
def enhanced_master_agent(query):
    response = master_agent(query)
    sentiment = sentiment_pipeline(query)[0]
    sent_label = sentiment['label']
    sent_score = sentiment['score']

    if sent_label == "NEGATIVE" and sent_score > 0.85:
        response += "\n\nI noticed you're upset. Would you like to escalate this to a human agent?"
    return response

In [12]:
iface = gr.Interface(
    fn=enhanced_master_agent,
    inputs=gr.Textbox(lines=2, placeholder="Enter your query here..."),
    outputs="text",
    title="Amazon Customer Support Assistant",
    description="A multi-agent AI system to help with returns, deliveries, technical issues, and general inquiries."
)

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://52849853500ec15a44.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
